In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [4]:
# selenium 3
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType

import os

In [61]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager

class TabelaResumoDataScraper:
    def __init__(self, setor_financeiro, options, service):
        self.setor_financeiro = setor_financeiro
        self.options = options
        self.service = service
        self.lista_dataframes = []
        self.iteracao = 0

    def navegador_get(self, acao):
        navegador = webdriver.Chrome(service=self.service, options=self.options)
        navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
        return navegador

    def obter_datas(self, navegador):
        while True:
            try:
                elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_bp"]/thead/tr/th[2]/select')
                datas = [elemento.text for elemento in elementos_data]
                if len(datas) == 1:
                    string_de_datas = datas[0]
                    datas = string_de_datas.split('\n')
                return datas
            except StaleElementReferenceException:
                continue

    def obter_dados_tabela(self, navegador, data):
        while True:
            try:
                select_element = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[2]/div[3]/table/thead/tr/th[2]/select')
                select = Select(select_element)
                select.select_by_visible_text(data)

                tabela = navegador.find_element(By.ID, 'pagina_empresa_bp')
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                lista_resumo_balanco = []
                for linha in linhas:
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_resumo_balanco.append(celula.text)

                return lista_resumo_balanco
            except StaleElementReferenceException:
                continue

    def processar_dados(self, acao, datas, lista_resumo_balanco):
        dados = {
            'datas': datas,
            'caixa_equivalentes_caixa': [np.nan] * len(datas),
            'ativo_total': [],
            'divida_curto_prazo': [np.nan] * len(datas),
            'divida_longo_prazo': [np.nan] * len(datas),
            'divida_bruta': [np.nan] * len(datas),
            'divida_liquida': [np.nan] * len(datas),
            'patrimonio_liquido': [],
            'valor_patrimonial_acao': [],
            'acoes_ordinarias': [],
            'acoes_preferenciais': [],
            'total': []
        }

        for i in range(len(datas)):
            dados['ativo_total'].append(lista_resumo_balanco[1].replace('R$','').replace(',','.').strip())
            dados['patrimonio_liquido'].append(lista_resumo_balanco[3].replace('R$','').replace(',','.').strip())
            dados['valor_patrimonial_acao'].append(lista_resumo_balanco[5].replace('R$','').replace(',','.').strip())
            dados['acoes_ordinarias'].append(lista_resumo_balanco[7].replace('R$','').replace(',','.').strip())
            dados['acoes_preferenciais'].append(lista_resumo_balanco[9].replace('R$','').replace(',','.').strip())
            dados['total'].append(lista_resumo_balanco[11].replace('R$','').replace(',','.').strip())

    
        df_resumo_balanco = pd.DataFrame(dados)
        
        if acao in self.setor_financeiro:
            colunas = ['ativo_total', 'patrimonio_liquido']
        else:
            colunas = ['caixa_equivalentes_caixa', 'ativo_total', 'divida_curto_prazo', 'divida_longo_prazo', 'divida_bruta', 'patrimonio_liquido']

        for col in colunas:
            df_resumo_balanco[col] = df_resumo_balanco[col].apply(self.converter_valor)

        df_resumo_balanco['tic'] = acao
        self.lista_dataframes.append(df_resumo_balanco)

        self.iteracao += 1
        if self.iteracao % 5 == 0:
            self.salvar_dados()

    def salvar_dados(self):
        pd.concat(self.lista_dataframes).to_csv('dados/resumo_balanco.csv')

    @staticmethod
    def converter_valor(valor):
        try:
            return float(valor)
        except ValueError:
            return np.nan




In [7]:
from selenium.webdriver.chrome.service import Service
refazer_comeco = True
chrome_driver_path = "/usr/bin/chromedriver"
service = Service(executable_path=chrome_driver_path)

options = Options()
options.add_argument("--headless")  # Executar em modo headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
options.add_argument("--disable-gpu")  # Desativar o uso de GPU
options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela

codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',
    #'VVBR3', 'BTOW3','ENBR3',GNDI3,IGTA3
    'BRFS3', 'BRKM5',  'CCRO3', 'CIEL3', 'CMIG4', 'COGN3', 'CPFE3',
    'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3',
     'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4',  'GOAU4', 'GOLL4', 'HAPV3',
     'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
    'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PRIO3', 'QUAL3',
    'RADL3', 'RAIL3', 'RENT3', 'SANB11', 'SBSP3','SUZB3', 'TAEE11', 'TIMS3', 'TOTS3',
    'UGPA3', 'USIM5', 'VALE3', 'VIVT3', 'WEGE3', 'WIZC3','YDUQ3','SOMA3','BPAC11','MDIA3'
]

In [66]:
# Uso da classe
setor_financeiro = {'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'ITUB4', 'BPAC11', 'ITUB4', 'SANB11', 'IRBR3'}

chrome_driver_path = "/usr/bin/chromedriver"
service = Service(executable_path=chrome_driver_path)

options = Options()
options.add_argument("--headless")  # Executar em modo headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
options.add_argument("--disable-gpu")  # Desativar o uso de GPU
options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela

scraper = TabelaResumoDataScraper(setor_financeiro, options, service)

navegador = scraper.navegador_get('BBAS3')

datas = scraper.obter_datas(navegador)

In [70]:
select_element = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[2]/div[3]/table/thead/tr/th[2]/select')
select = Select(select_element)
select.select_by_visible_text(datas[1])
tabela = navegador.find_element(By.ID, 'pagina_empresa_bp')
linhas = tabela.find_elements(By.TAG_NAME, 'tr')
lista_resumo_balanco = []
for linha in linhas:
    celulas = linha.find_elements(By.TAG_NAME, 'td')
    for celula in celulas:
        lista_resumo_balanco.append(celula.text)

In [71]:
lista_resumo_balanco

[]

In [ ]:
select_element = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[2]/div[3]/table/thead/tr/th[2]/select')
select = Select(select_element)
WebDriverWait(navegador, 10).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[2]/div[3]/table/thead/tr/th[2]/select'))
                )
select.select_by_visible_text(datas[1])

tabela = navegador.find_element(By.ID, 'pagina_empresa_bp')
linhas = tabela.find_elements(By.TAG_NAME, 'tr')

lista_resumo_balanco = []
for linha in linhas:
    celulas = linha.find_elements(By.TAG_NAME, 'td')
    for celula in celulas:
        lista_resumo_balanco.append(celula.text)
lista_resumo_balanco